<a href="https://colab.research.google.com/github/walexand3r/4YP/blob/master/CIFAR10/ML/hypoML%20Testing/v1.0_LastActiv(Add).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Forming 2 networks, one for D and one for +, combining by adding before final activation layer

Model Version 1.0

Added: 

- Made all processes into functions, making the four tests easier to run





In [0]:

# This block imports relevant packages and sets intial seed to ensure reproducability
# Seed value
seed_value= 0

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os, datetime
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
%tensorflow_version 2.x
import tensorflow as tf
tf.random.set_seed(seed_value)
# for later versions: tf.random.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
import tensorflow.keras as keras
# from tensorflow.keras import backend as K
# session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
# K.set_session(sess)
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Dense, Dropout, Flatten,  Reshape, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LeakyReLU, Softmax, Concatenate
from tensorflow.keras.layers import Maximum, Add
from tensorflow import math

%load_ext tensorboard
import pandas as pd

# Mount drive

from google.colab import drive
drive.mount('/content/drive/')

In [0]:
# Sets paths for train/test D/D+
folder = 'RGBHSV/32x64'
path = '/content/drive/My Drive/4th Year/4YP/CIFAR10/Processed Datasets/'

# Sets results path
saveFolder = 'LastActivation/Add/'
savePath = '/content/drive/My Drive/4th Year/4YP/CIFAR10/Results/hypoML Testing/' + saveFolder

In [0]:
def createModel():
  batch_size = 64
  epochs = 20
  num_classes = 10

  # Forming 2 input models, 'left' and 'right'
  # left
  leftInputs = keras.Input(shape=(32, 32, 3))
  leftx = Conv2D(64, kernel_size = (3,3), padding = 'same')(leftInputs)
  leftx = LeakyReLU(alpha = 0.3)(leftx)

  leftx = Conv2D(64, kernel_size=(3, 3), padding = 'same')(leftx)
  leftx = LeakyReLU(alpha = 0.3)(leftx)
  leftx = MaxPooling2D(pool_size=(2,2))(leftx)

  leftx = Conv2D(128, kernel_size=(3, 3), padding = 'same')(leftx)
  leftx = LeakyReLU(alpha = 0.3)(leftx)

  leftx = Conv2D(128, kernel_size=(3, 3), padding = 'same')(leftx)
  leftx = LeakyReLU(alpha = 0.3)(leftx)
  leftx = MaxPooling2D(pool_size=(2,2))(leftx)

  leftx = Conv2D(256, kernel_size=(3, 3), padding = 'same')(leftx)
  leftx = LeakyReLU(alpha = 0.3)(leftx)

  leftx = Conv2D(256, kernel_size=(1, 1), padding = 'same')(leftx)
  leftx = LeakyReLU(alpha = 0.3)(leftx)
  leftx = Dropout(rate = 0.5)(leftx)
  leftx = Conv2D(10, kernel_size=(3, 3), padding = 'same')(leftx)
  leftx = LeakyReLU(alpha = 0.3)(leftx)
  leftx = tf.reduce_mean(leftx,(1,2))

  # right
  rightInputs = keras.Input(shape=(32, 32, 3))
  rightx = Conv2D(64, kernel_size = (3,3), padding = 'same')(rightInputs)
  rightx = LeakyReLU(alpha = 0.3)(rightx)

  rightx = Conv2D(64, kernel_size=(3, 3), padding = 'same')(rightx)
  rightx = LeakyReLU(alpha = 0.3)(rightx)
  rightx = MaxPooling2D(pool_size=(2,2))(rightx)

  rightx = Conv2D(128, kernel_size=(3, 3), padding = 'same')(rightx)
  rightx = LeakyReLU(alpha = 0.3)(rightx)

  rightx = Conv2D(128, kernel_size=(3, 3), padding = 'same')(rightx)
  rightx = LeakyReLU(alpha = 0.3)(rightx)  
  rightx = MaxPooling2D(pool_size=(2,2))(rightx)

  rightx = Conv2D(256, kernel_size=(3, 3), padding = 'same')(rightx)
  rightx = LeakyReLU(alpha = 0.3)(rightx)

  rightx = Conv2D(256, kernel_size=(1, 1), padding = 'same')(rightx)
  rightx = LeakyReLU(alpha = 0.3)(rightx)
  rightx = Dropout(rate = 0.5)(rightx)
  rightx = Conv2D(10, kernel_size=(3, 3), padding = 'same')(rightx)
  rightx = LeakyReLU(alpha = 0.3)(rightx)
  rightx = tf.reduce_mean(rightx,(1,2))

  # merging
  x = Add()([leftx,rightx])
  outputs = Softmax()(x)

  model = keras.Model(inputs=(leftInputs, rightInputs), outputs=outputs, name='cifar10_model')

  model.compile(loss = keras.losses.categorical_crossentropy,
                optimizer = keras.optimizers.Adam(learning_rate = 0.001, beta_1 = 0.98),
                metrics=['accuracy'])

  model.summary()
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  return model, logdir, tensorboard_callback

In [0]:
def createRawData(xTestL,yTestL,xTestR,model):
  # This function makes predictions on test data, compares to real class labels
  # and constructs output data to display this

  n_test = len(yTestL)
  yTestL = np.argmax(yTestL,axis=1)
  print('Making Predictions')
  # Make predictions
  probs = model.predict([xTestL,xTestR], 
                        batch_size=batch_size,
                        verbose=1,)
  predictions = np.argmax(probs,axis = 1)
  # Forming into required raw data format 
  Correctness = np.zeros(n_test)
  for i in range(len(predictions)):
    if np.array_equal(predictions[i],yTestL[i]):
      Correctness[i] = True
    else:
      Correctness[i] = False

  GroundTruth= yTestL
  RawData = np.zeros((n_test,5))
  RawData[:,1] = GroundTruth
  RawData[:,2] = predictions
  RawData[:,4] = Correctness

  for i in range(n_test):
    RawData[i,0] = i+1
  #print(sum(Correctness))
  return RawData

In [0]:
def importData(trainingOn,testingOn):
  # Data Import
  # All values are scaled to range 0-1
  print('Loading Datasets')

  ### Left data 
  # Whatever test is being run, left data is always cifar10 RGB
  (xTrainL, yTrainL), (xTestL, yTestL) = cifar10.load_data()
  n_train = np.size(xTrainL,0)
  n_test = np.size(xTestL,0)
  num_classes = 10

  xTrainL = xTrainL / 255
  xTestL = xTestL / 255
 
  # One-hot encoding class labels
  yTrainL = keras.utils.to_categorical(yTrainL, num_classes)
  yTestL = keras.utils.to_categorical(yTestL, num_classes)

  # input image dimensions
  row_length = np.size(xTrainL,1) * np.size(xTrainL,2) * np.size(xTrainL,3) + 1
  img_rows, img_cols = np.size(xTrainL,1), np.size(xTrainL,2)

  ### Right data
  # Right data depends on what test is being run
  print('Importing csv data')
  if trainingOn == 'M':
    loadPath = path + folder + '/+/+_noise_train.csv'
    TrainR = pd.read_csv(loadPath,header=None,engine='python')
  elif trainingOn == 'M+':
    loadPath = path + folder + '/+/+_train.csv'
    TrainR = pd.read_csv(loadPath,header=None,engine='python')
  else:
    print('Invalid input')

  if testingOn == 'D':
    loadPath = path + folder + '/+/+_noise_test.csv'
    TestR = pd.read_csv(loadPath,header=None,engine='python')
  elif testingOn == 'D+':
    loadPath = path + folder + '/+/+_test.csv'
    TestR = pd.read_csv(loadPath,header=None,engine='python')
  else:
    print('Invalid input')
  print('Done')

  # Extracting data
  yTestR = TestR.iloc[:,0]
  xTestR = TestR.iloc[:,1:]
  yTrainR = TrainR.iloc[:,0]
  xTrainR = TrainR.iloc[:,1:]
  xTrainR = xTrainR.values.reshape(n_train,img_rows,img_cols,3)
  xTestR = xTestR.values.reshape(n_test,img_rows,img_cols,3)

  # convert class vectors to binary class matrices
  yTrainR = keras.utils.to_categorical(yTrainR, num_classes)
  yTestR = keras.utils.to_categorical(yTestR, num_classes)

  print('Data created')

  return xTrainL,yTrainL,xTrainR,yTrainR,xTestL,yTestL,xTestR,yTestR


In [0]:
# Test 1: R(M,D)     

batch_size = 64
epochs = 50
num_classes = 10
(xTrainL,yTrainL,xTrainR,yTrainR,xTestL,yTestL,xTestR,yTestR) = importData('M','D')

model, logdir, tensorboard_callback = createModel()

model.fit((xTrainL,xTrainR),
          yTrainL,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split = 0.02,
          callbacks = [tensorboard_callback])

RawData_1 = createRawData(xTestL,yTestL,xTestR,model)

print('Saving')
# Saving
savePath1 = savePath + 'R(M,D).csv'
pd.DataFrame(RawData_1).to_csv(savePath1,header=None, index=None)

# test_scores = model.evaluate([xTestL,xTestR], yTestL, verbose=2)
# print('Test loss:', test_scores[0])
# print('Test accuracy:', test_scores[1])
print(sum(RawData_1[:,4]))

In [0]:
# Test 2: R(M,D+)     

batch_size = 64
epochs = 50
num_classes = 10
(xTrainL,yTrainL,xTrainR,yTrainR,xTestL,yTestL,xTestR,yTestR) = importData('M','D+')

model, logdir, tensorboard_callback = createModel()

model.fit((xTrainL,xTrainR),
          yTrainL,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split = 0.02,
          callbacks = [tensorboard_callback])

RawData_2 = createRawData(xTestL,yTestL,xTestR,model)

print('Saving')
# Saving
savePath2 = savePath + 'R(M,D+).csv'
pd.DataFrame(RawData_2).to_csv(savePath2,header=None, index=None)

# test_scores = model.evaluate([xTestL,xTestR], yTestL, verbose=2)
# print('Test loss:', test_scores[0])
# print('Test accuracy:', test_scores[1])
print(sum(RawData_2[:,4]))

In [0]:
# Test 3: R(M+,D)     

batch_size = 64
epochs = 50
num_classes = 10
(xTrainL,yTrainL,xTrainR,yTrainR,xTestL,yTestL,xTestR,yTestR) = importData('M+','D')

model, logdir, tensorboard_callback = createModel()

model.fit((xTrainL,xTrainR),
          yTrainL,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split = 0.02,
          callbacks = [tensorboard_callback])

RawData_3 = createRawData(xTestL,yTestL,xTestR,model)

print('Saving')
# Saving
savePath3 = savePath + 'R(M+,D).csv'
pd.DataFrame(RawData_3).to_csv(savePath3,header=None, index=None)

# test_scores = model.evaluate([xTestL,xTestR], yTestL, verbose=2)
# print('Test loss:', test_scores[0])
# print('Test accuracy:', test_scores[1])
print(sum(RawData_3[:,4]))

In [0]:
# Test 4: R(M+,D+)     

batch_size = 64
epochs = 50
num_classes = 10
(xTrainL,yTrainL,xTrainR,yTrainR,xTestL,yTestL,xTestR,yTestR) = importData('M+','D+')

model, logdir, tensorboard_callback = createModel()

model.fit((xTrainL,xTrainR),
          yTrainL,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split = 0.02,
          callbacks = [tensorboard_callback])

RawData_4 = createRawData(xTestL,yTestL,xTestR,model)

print('Saving')
# Saving
savePath4 = savePath + 'R(M+,D+).csv'
pd.DataFrame(RawData_4).to_csv(savePath4,header=None, index=None)

# test_scores = model.evaluate([xTestL,xTestR], yTestL, verbose=2)
# print('Test loss:', test_scores[0])
# print('Test accuracy:', test_scores[1])
print(sum(RawData_4[:,4]))

In [0]:
!kill 725
%tensorboard --logdir logs  # start Tensorboard
